In [ ]:
## assignment - 
# https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [17]:
import pandas as pd
import numpy as np

In [18]:
reviews = pd.read_csv("IMDB Dataset.csv")

In [19]:
reviews

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [20]:
reviews.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [21]:
reviews.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [22]:
reviews.shape

(50000, 2)

In [ ]:
import re
from nltk.corpus import stopwords

# Get the English stop words
stop_words = set(stopwords.words('english'))

def clean_text(text):
    """
    This function cleans the input text by performing the following steps:
    1. Removes HTML tags.
    2. Removes punctuation and special characters.
    3. Converts text to lowercase.
    4. Tokenizes the text and removes stopwords.
    """
    
    # 1. Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # 2. Remove punctuation and special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    
    # 3. Convert to lowercase
    text = text.lower()
    
    # 4. Remove stopwords
    # Tokenize the text and remove stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    
    return ' '.join(tokens)

# Apply the cleaning function to the 'review' column
reviews['cleaned_review'] = reviews['review'].apply(clean_text)

print("Original Review:")
print(reviews['review'][0])
print("\nCleaned Review:")
print(reviews['cleaned_review'][0])

In [ ]:
## preprocessing